# Applied Data Science Capstone Notebook

### This is my Capstone Project for the Applied Data Science Capstone course taught by IBM on Coursera.

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import geocoder
import json
from pandas.io.json import json_normalize
print('Importing done!')

Importing done!
